In [1]:
import numpy as np
import pandas as pd
import pickle
import janestreet
import cupy as cp
import gc

from time import time
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler, StandardScaler, RobustScaler, PowerTransformer, MaxAbsScaler, Normalizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import optimizers
from multiprocessing.pool import ThreadPool
from tensorflow.keras.models import load_model

In [2]:
def getDf():
    df = pd.read_csv('../input/jane-street-market-prediction/train.csv')
    df = df.dropna()
    df = df.drop(columns=['resp_1','resp_2','resp_3','resp_4','ts_id'])
    
    return df
            
def getTVT_split(split,mn):
    global df
    y = np.where(df['resp'] > mn,1,0).reshape(len(df),1)
    resp = df['resp'].values.reshape(-1,1)
    x = df.drop(columns=['resp']).values
    date = x[:,:1]
    weight = x[:,1:2]
    x = x[:,2:]
    
    x_train, x_test = train_test_split(x,train_size=split,random_state=5)

    y_train, y_test = train_test_split(y,train_size=split,random_state=5)
    
    gc.collect()
    
    return x_train, x_test, y_train, y_test
    
def scaling(i):
    global x_train, x_test, scalers
    if(scalers[i]!="no"):
        x_train[:,i+0:i+1] = scalers[i].fit_transform(x_train[:,i+0:i+1])
        x_test[:,i+0:i+1] = scalers[i].transform(x_test[:,i+0:i+1])
    gc.collect()

In [3]:
df = getDf()

In [4]:
split = 0.57165
mn = 0
hid_layers = 5
act_in = 'elu'
nn_in = 95
act_out = 'linear'
nn_hid = [72,54,60,6,83]
act_hid = ['softplus','tanh','softsign','relu','selu']
scalers = [MinMaxScaler(), MaxAbsScaler(), StandardScaler(), 'no', StandardScaler(with_std=False),
           StandardScaler(with_mean=False, with_std=False), StandardScaler(with_mean=False),
           RobustScaler(), RobustScaler(with_centering=False), RobustScaler(), 'no', 
           StandardScaler(with_mean=False, with_std=False), 'no', QuantileTransformer(random_state=5),
           MaxAbsScaler(), RobustScaler(with_centering=False), PowerTransformer(standardize=False),
           StandardScaler(with_std=False), StandardScaler(with_std=False), 'no', MinMaxScaler(),
           RobustScaler(with_scaling=False), PowerTransformer(),
           StandardScaler(with_mean=False, with_std=False), RobustScaler(with_scaling=False), 'no',
           QuantileTransformer(random_state=5), 'no', StandardScaler(with_mean=False),
           RobustScaler(with_centering=False), StandardScaler(with_std=False),
           StandardScaler(with_mean=False, with_std=False), StandardScaler(with_std=False),
           StandardScaler(), QuantileTransformer(random_state=5), StandardScaler(),
           QuantileTransformer(random_state=5), StandardScaler(with_mean=False, with_std=False),
           RobustScaler(with_centering=False), MinMaxScaler(feature_range=(-1, 1)), PowerTransformer(),
           PowerTransformer(standardize=False), QuantileTransformer(random_state=5),
           RobustScaler(with_centering=False),
           QuantileTransformer(output_distribution='normal', random_state=5),
           MinMaxScaler(feature_range=(-1, 1)), StandardScaler(with_mean=False, with_std=False),
           StandardScaler(), RobustScaler(), 'no', RobustScaler(with_scaling=False),
           StandardScaler(with_std=False), RobustScaler(with_centering=False, with_scaling=False),
           StandardScaler(with_mean=False, with_std=False),
           RobustScaler(with_centering=False, with_scaling=False), MaxAbsScaler(), StandardScaler(),
           MaxAbsScaler(), QuantileTransformer(random_state=5), MaxAbsScaler(),
           QuantileTransformer(random_state=5), StandardScaler(with_mean=False),
           StandardScaler(with_mean=False), RobustScaler(with_centering=False),
           StandardScaler(with_mean=False), PowerTransformer(), RobustScaler(with_centering=False),
           MinMaxScaler(feature_range=(-1, 1)),
           QuantileTransformer(output_distribution='normal', random_state=5),
           RobustScaler(with_centering=False), PowerTransformer(), MinMaxScaler(), RobustScaler(),
           PowerTransformer()]
n_features = len(scalers)
ls = 'mean_squared_error'
opt = 'adam'

x_train, x_test, y_train, y_test = getTVT_split(split,mn)
gc.collect()

f_used = [0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
          1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
          1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
          0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 
          1, 0]
x_train = x_train[:,np.nonzero(f_used)][:,0]
x_test = x_test[:,np.nonzero(f_used)][:,0]

p = ThreadPool()
p.map(scaling, range(n_features))
p.close()
p.join()
gc.collect()
eps = 144
batch = 7208
features_used = np.array(np.nonzero(f_used))[0]

In [5]:
model = Sequential()
model.add(Dense(95,input_dim=(74),activation='elu'))
model.add(Dense(72,activation='softplus'))
model.add(Dense(54,activation='tanh'))
model.add(Dense(60,activation='softsign'))
model.add(Dense(6,activation='relu'))
model.add(Dense(83,activation='selu'))
model.add(Dense(y_train.shape[1],activation='linear'))

model.compile(loss=ls,optimizer=opt)

model.fit(x_train,y_train,epochs=eps,batch_size=batch,verbose=0)

In [6]:
print("Train size: {}\n\nLower limit for earning: {}\n\nNumber of hiden layers: {}\n\nActivation on input layer: {}\n\nNumber of neurons on input layer: {}\n\nActivation on output layer: {}\n\nNumber of neurons on hidden layers: {}\n\nActivation on hidden layers: {}\n\nFeatures used: {}\n\nScalers of the features: {}\n\nLoss of the model: {}\n\nOptimizer of the model: {}\n\nEpochs of training: {}\n\nBatch size: {}".format(split,
mn,hid_layers,act_in,nn_in,act_out,nn_hid,act_hid,features_used,scalers,ls,opt,eps,batch))

Train size: 0.57165

Lower limit for earning: 0

Number of hiden layers: 5

Activation on input layer: elu

Number of neurons on input layer: 95

Activation on output layer: linear

Number of neurons on hidden layers: [72, 54, 60, 6, 83]

Activation on hidden layers: ['softplus', 'tanh', 'softsign', 'relu', 'selu']

Features used: [  1   2   4   5   6   7   8  13  17  18  19  22  23  25  26  27  29  32
  34  35  37  38  40  41  43  44  50  52  54  57  58  61  62  63  64  68
  69  70  73  75  77  79  81  82  84  87  88  91  92  93  94  97  99 100
 101 102 104 105 106 107 108 111 114 115 116 117 118 120 122 123 124 126
 127 128]

Scalers of the features: [MinMaxScaler(), MaxAbsScaler(), StandardScaler(), 'no', StandardScaler(with_std=False), StandardScaler(with_mean=False, with_std=False), StandardScaler(with_mean=False), RobustScaler(), RobustScaler(with_centering=False), RobustScaler(), 'no', StandardScaler(with_mean=False, with_std=False), 'no', QuantileTransformer(random_state=5), Ma

In [7]:
env = janestreet.make_env()
iter_test = env.iter_test()
test_df = 0
p = ThreadPool()

In [8]:
def scaler(i):
    global test_df, scalers
    if(scalers[i]!="no"):
        test_df[:,i+0:i+1] = scalers[i].transform(test_df[:,i+0:i+1])

In [9]:
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df.drop(columns=['date','weight','feature_0','feature_3','feature_9','feature_10',
                                        'feature_11','feature_12','feature_14','feature_15','feature_16',
                                        'feature_20','feature_21','feature_24','feature_28','feature_30',
                                        'feature_31','feature_33','feature_36','feature_39','feature_42',
                                        'feature_45','feature_46','feature_47','feature_48','feature_49',
                                        'feature_51','feature_53','feature_55','feature_56','feature_59',
                                        'feature_60','feature_65','feature_66','feature_67','feature_71',
                                        'feature_72','feature_74','feature_76','feature_78','feature_80',
                                        'feature_83','feature_85','feature_86','feature_89','feature_90',
                                        'feature_95','feature_96','feature_98','feature_103','feature_109',
                                        'feature_110','feature_112','feature_113','feature_119','feature_121',
                                        'feature_125','feature_129']).values
    if(np.isnan(test_df).any()):
        sample_prediction_df.action = 0
        env.predict(sample_prediction_df)
    else:
        p.map(scaler,range(74))
        sample_prediction_df.action = int(model(test_df,training=False).numpy()[0,0] > 0.5)
        env.predict(sample_prediction_df)